In [ ]:
%%bash
# If you are on Google Colab, this sets up everything needed.
# If not, you will want to pip install the cs7150lib as shown below.
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
pip install git+https://github.com/cs7150/cs7150lib@main

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CS7150/cs7150lib/blob/main/notebooks/LearningAFourLayerPerceptron.ipynb)

# Learning a four-layer perceptron

We learned in Lecture 1 that Rosenblatt's Perceptron was limited because it only learned one layer.  It was a *shallow* learning architecture.

We want to learn *deep* layers.

To do it, we start with the simple idea from [Rumelhart 1986](https://papers.baulab.info/Rumelhart-1986.pdf): use *derivatives* to figure out the updates.

But, to make this work, there are a lot of details that need to be done right.  Fortunately our CPUs run a lot faster than Rosenblatt's and Rumelharts.  We can do experiments quickly, and we will explore the important details in this notebook.  You do not need a GPU for this exercise.

This notebook is designed to be able to work on Google colab.


In [ ]:
import torch, numpy
from collections import OrderedDict
from copy import deepcopy
from torch.nn import Sequential, Linear, ReLU, Sigmoid, Tanh
from matplotlib import pyplot as plt
from cs7150 import seeded_net, non_linearly_separable_data, MLPHistoryWidget, Sign
from baukit import show, pbar

# Adding more layers to Rosenblatt's setup

The original perceptron would have had one trainable neuron with an input weight for the horizontal position of a dot, and a second input weight for the vertical position of the dot.  In other words, it would have had a two-dimenaional input.

We will make the setup more powerful by adding three more trainable layers before this last neuron.  Each layer will take two inputs and produce two outputs, as illustrated here:

<img src="https://cs7150.baulab.info/2022-Fall/colab/mlp-architecture.png">

The code is below.

## Exercise 1.  Run the multilayer perceptron.

Run the multilayer perceptron below, without changing any code.

Look at the training curve after 1000 steps.  What (doesn't) happen?

That is because the derivatives are always zero.  You will fix this in Exercise 2.

In [ ]:
# For Exercise 1, just read and run the code.
data, labels = non_linearly_separable_data()

mlp = torch.nn.Sequential(OrderedDict([
    ('layer1', Sequential(Linear(2, 2), Sign())),
    ('layer2', Sequential(Linear(2, 2), Sign())),
    ('layer3', Sequential(Linear(2, 2), Sign())),
    ('layer4', Sequential(Linear(2, 1), Sign()))
]))
seeded_net(1, mlp) # Seed the parameters pseudorandomly

history = []
learning_rate = 0.01
for iteration in pbar(range(1000)):
    preds = mlp(data)[:,0]
    loss = ((preds - labels) ** 2).mean()
    grads = torch.autograd.grad(loss, mlp.parameters())
    with torch.no_grad():
        accuracy = (preds.sign() == labels).sum().item() / len(labels)
        # Remember a history of network weights, losses, and accuracies
        history.append((deepcopy(mlp), dict(loss=loss.item(), accuracy=accuracy)))
        # Update the network parameters
        for (name, parameter), grad in zip(mlp.named_parameters(), grads):
            parameter -= learning_rate * grad
# Visualize the history.
MLPHistoryWidget(data=data, labels=labels, history=history)

## Exercise 2.  Make the network differentiable using Tanh().

Copy the Exercise 1 code into the cell below, and then edit it so that instead of using the Sign() step nonlinearity, the network uses the Tanh() sigmoid nonlinearity.  Try things again.  What's the difference?

What accuracy is achieved in 1000 training iterations?

In [ ]:
# Your code for Exercise 2 here.

## Exercise 3. Speed up training with a faster learning rate.

Find a faster learning rate that speeds up training.

Your goal is to achieve 99% accuracy in 300 training iterations.  Is it possible?

How does your training procedure work if you change the seed, e.g., `seeded_net(2, mlp)` instead of `seeded_net(1, mlp)`?

In [ ]:
# Your code for Exercise 3 here.

## Exercise 4.  Monitor the gradients

Copy the code again below, and use the seed of 2, or any other seed which is getting stuck for you.

Use code like the following to create a add a list of gradients to the training history. 

```
        gradlog = {name + ' grad': grad.norm().item()
                   for (name, _), grad in zip(mlp.named_parameters(), grads) if 'weight' in name}
        history.append((deepcopy(mlp), dict(loss=loss.item(), accuracy=accuracy, **gradlog)))
```

Do you notice any trends with the gradients during training?   What happens when your training gets stuck?

Sometimes, some layers have very different gradients than other layers.

In [ ]:
# Your code for Exercise 4 here.

## Exercise 5. Experiment with very deep networks and weight initialization.

[Kaiming He (2015)](https://papers.baulab.info/He-2015.pdf) and [Xavier Glorot (2010)](https://papers.baulab.info/Glorot-2010.pdf) wrote famous papers where they noticed that the scale of the weight initialization was often critical to successful training when the network is deeper than a few layers.  The `seeded_net` function initializes parameters with unit variance, but you can change this.

Do the following:

   1. Make the network 6 layers deep or deeper by copying the early layers and giving them new names.
   2. Monitor gradients as in exercise 4 above
   3. Experiment with larger and smaller weights by adding code like this after `seeded_net`:

```
seeded_net(1, mlp) # Seed the parameters pseudorandomly
with torch.no_grad():
    for parameter in mlp.parameters():
        parameter *= 0.7
```

What is the impact of initialization on initial gradients at iteration 0?

Typically you either have **vanishing gradients** or **exploding gradients** unless initialization is just right.  Which are you seeing with this architecture?

The famous [AlexNet network, Krizhevsky (2012)](https://papers.baulab.info/Krizhevsky-2012.pdf) was 8 layers deep.  Can you get an 8-layer-deep version of this network to train?

In [ ]:
# Your code for Exercise 5 here.

## Exercise 6.  Experiment with wide and shallow networks.

[George Cybenko (1989)](https://papers.baulab.info/Cybenko-1989.pdf) and
[Kurt Hornik (1991)](https://papers.baulab.info/also/Hornik-1991.pdf)
wrote famous papers that showed that any continuous function can be approximated using
just a couple layers, if you use enough neurons.

Try training a network with just two layers. How many neurons do you need to get good accuracy?

[Dauphin (2014)](https://papers.baulab.info/also/Dauphin-2014.pdf) has argued that, in practice, when you have a high dimensional network, you do not run into local minima as often (he argues that the main problem is flat saddle points instead).

Do you see any other phenomenon in the shape of the learned function of a wide shallow network that bothers you?

In [ ]:
# Your code for Exercise 6 here.

## Exercise 7.  Experiment with networks that are both wide and shallow.

Classical machine learning theory would suggest that the problems you saw in Exercise 6 with very shallow networks with many neurons would get even worse if you added even more neurons on more layers.

However, classical theory does not explain the success of deep networks in this setting.  See [Nakkiran 2019](https://papers.baulab.info/also/Nakkiran-2019.pdf) for a discussion of some of the issues.

Try it out.  Create a network with 8 layers, with 10 hidden dimensions at every layer (except the input of the network should remain 2 dimensional and the output should be one dimensional).

Find a weight initialization and learning rate that seems to be stable.

* Does the network converge to an accurate solution for different weights?
* How does the learned function compare to the functions learned in Exercise 6?





In [ ]:
# Your code for Exercise 7 here.